<a href="https://colab.research.google.com/github/dlbkv/AI-Step/blob/master/module7/lab24_recommendation_system2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Завдання

* замініть усі пропущені значення на пусті рядки
* отримайте список з назвами фільмів
* застосуйте TF-iDF до колонки `description`
* на основі отриманих даних побудуйте таблицю з коефіцієнтами подібності
* напишіть функцію яка отримує як аргемент назву фільму та число і повертає відповідну кількість найбільш схожих фільмів

In [34]:
import pandas as pd

df = pd.read_csv("https://raw.githubusercontent.com/HalyshAnton/IT-Step-Pyton-AI/main/module7/data/netflix_titles.csv")

df.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm..."
1,s2,TV Show,Blood & Water,NaN,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t..."
2,s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",NaN,"September 24, 2021",2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...
3,s4,TV Show,Jailbirds New Orleans,NaN,NaN,NaN,"September 24, 2021",2021,TV-MA,1 Season,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo..."
4,s5,TV Show,Kota Factory,NaN,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...


In [35]:
df.fillna(value='', inplace=True)

titles = df['title']

full_descriptions = df['f_descriptions'] = df[['cast', 'listed_in', 'description']].apply(lambda row: ' '.join(row.values), axis=1)
full_descriptions

0        Documentaries As her father nears the end of ...
1       Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...
2       Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...
3        Docuseries, Reality TV Feuds, flirtations and...
4       Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...
                              ...                        
8802    Mark Ruffalo, Jake Gyllenhaal, Robert Downey J...
8803     Kids' TV, Korean TV Shows, TV Comedies While ...
8804    Jesse Eisenberg, Woody Harrelson, Emma Stone, ...
8805    Tim Allen, Courteney Cox, Chevy Chase, Kate Ma...
8806    Vicky Kaushal, Sarah-Jane Dias, Raaghav Chanan...
Length: 8807, dtype: object

In [36]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
import string
from nltk.corpus import stopwords
nltk.download("stopwords")
nltk.download('punkt')
nltk.download('wordnet')

class MyTokenizer:
     def __init__(self, stemmatize=False, lemmatize=False, stop_words=False):
         self.stemmatize = stemmatize
         self.lemmatize = lemmatize
         self.stop_words = stop_words

     def __call__(self, text):
      words = word_tokenize(text)
      if self.stop_words:
        stop_words = set(stopwords.words("english"))
        words = [word for word in words if word.lower() not in stop_words]

      words = [word for word in words if word.lower() not in string.punctuation]

      if self.stemmatize:
        stemmer = PorterStemmer()
        words = [stemmer.stem(word.lower()) for word in words]

      if self.lemmatize:
        lemmatizer = WordNetLemmatizer()
        words = [lemmatizer.lemmatize(word.lower()) for word in words]

      return words

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [41]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

tokenizer = MyTokenizer(stemmatize=True, lemmatize=True, stop_words=True)
vectorizer = TfidfVectorizer(max_features=5000,tokenizer=tokenizer)

data = vectorizer.fit_transform(full_descriptions)

sim_matrix  = cosine_similarity(data, data)

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [42]:
df_sim = pd.DataFrame(sim_matrix,
                      columns=titles,
                      index=titles)

df_sim.head()

title,Dick Johnson Is Dead,Blood & Water,Ganglands,Jailbirds New Orleans,Kota Factory,Midnight Mass,My Little Pony: A New Generation,Sankofa,The Great British Baking Show,The Starling,...,Zak Storm,Zed Plus,Zenda,Zindagi Gulzar Hai,Zinzana,Zodiac,Zombie Dumb,Zombieland,Zoom,Zubaan
title,,,,,,,,,,,,,,,,,,,,,
Dick Johnson Is Dead,1.000000,0.000000,0.000000,0.000000,0.019868,0.000000,0.000000,0.000000,0.038694,0.047400,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.018608,0.070177
Blood & Water,0.000000,1.000000,0.075175,0.019802,0.073614,0.114324,0.038263,0.010326,0.047320,0.004135,...,0.065617,0.007347,0.097241,0.097390,0.010052,0.00422,0.123138,0.031262,0.000000,0.008433
Ganglands,0.000000,0.075175,1.000000,0.019718,0.088673,0.046688,0.064050,0.004461,0.055392,0.000000,...,0.018877,0.003174,0.004127,0.109892,0.004343,0.01423,0.093439,0.024022,0.009794,0.013688
Jailbirds New Orleans,0.000000,0.019802,0.019718,1.000000,0.069713,0.016651,0.000000,0.000000,0.065516,0.014070,...,0.006732,0.000000,0.000000,0.023929,0.000000,0.00000,0.025956,0.000000,0.015644,0.000000
Kota Factory,0.019868,0.073614,0.088673,0.069713,1.000000,0.045719,0.018195,0.004368,0.054242,0.016636,...,0.018485,0.032662,0.004042,0.155755,0.059322,0.00000,0.099374,0.030485,0.052073,0.003568


In [43]:
def get_recommendations(title, num_recommend):
  sim_titles = list(df_sim[title].nlargest(num_recommend).index)[1:]
  return sim_titles

In [44]:
title = 'Blood & Water'
k = 5

get_recommendations(title, k)

['The Underwear',
 'Intersection',
 'Miss Culinary',
 'Elite Short Stories: Nadia Guzmán']